In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('home_price_first_encode.csv')
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20131 entries, 0 to 20130
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Net_Metrekare       20131 non-null  int64  
 1   Brüt_Metrekare      20131 non-null  float64
 2   Oda_Sayısı          20131 non-null  int64  
 3   Bulunduğu_Kat       20131 non-null  object 
 4   Eşya_Durumu         20131 non-null  int64  
 5   Binanın_Yaşı        20131 non-null  float64
 6   Isıtma_Tipi         20131 non-null  object 
 7   Fiyat               20131 non-null  float64
 8   Şehir               20131 non-null  object 
 9   Binanın_Kat_Sayısı  20131 non-null  int64  
 10  Kullanım_Durumu     20131 non-null  int64  
 11  Yatırıma_Uygunluk   20131 non-null  int64  
 12  Takas               20131 non-null  int64  
 13  Tapu_Durumu         20131 non-null  object 
 14  Banyo_Sayısı        20131 non-null  float64
dtypes: float64(4), int64(7), object(4)
memory usage: 2.3+

In [6]:
import pandas as pd
import numpy as np

def review_and_optionally_delete(df, condition, description):


    outliers = df[condition]

    if outliers.empty:
        print(f"\nNo outliers found for: {description}")
        return df
    
    print(f"\n==============================")
    print(f"OUTLIERS FOUND: {description}")
    print(f"Count: {len(outliers)}")
    print(outliers.head(10))  # show first 10 outliers
    print("==============================")

    choice = input("Delete these rows? (y/n): ").strip().lower()
    
    if choice == "y":
        print("Deleting rows...\n")
        df = df[~condition]
    else:
        print("Keeping all rows.\n")

    return df

# Make a working copy
clean_df = df.copy()


condition = ~clean_df["Net_Metrekare"].between(10, 2000)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Net_Metrekare (<10 or >2000)")


condition = ~clean_df["Brüt_Metrekare"].between(20, 4000)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Brüt_Metrekare (<20 or >4000)")


condition = clean_df["Net_Metrekare"] > clean_df["Brüt_Metrekare"]
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Net_Metrekare greater than Brüt_Metrekare")


condition = ~clean_df["Oda_Sayısı"].between(0.5, 15)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Oda_Sayısı (not between 0.5 and 15)")

condition = ~clean_df["Banyo_Sayısı"].between(0, 10)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Banyo_Sayısı (not between 0 and 10)")

condition = clean_df["Banyo_Sayısı"] > clean_df["Oda_Sayısı"] + 2
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Banyo_Sayısı > Oda_Sayısı + 2")

# Negative or too small price
condition = clean_df["Fiyat"] <= 300000
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Fiyat ≤ 300000 TL (incorrect low price)")

condition = clean_df["Fiyat"] >= 500_000_000
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Fiyat ≥ 500M TL (unrealistic high price)")

clean_df["Binanın_Yaşı"] = pd.to_numeric(clean_df["Binanın_Yaşı"], errors="coerce")
condition = ~clean_df["Binanın_Yaşı"].between(0, 150)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Binanın_Yaşı not between 0 and 150")

condition = ~clean_df["Binanın_Kat_Sayısı"].between(1, 200)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Binanın_Kat_Sayısı (not between 1 and 200)")

output_filename = "outlier_check.csv"
clean_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"\n✓ Cleaned data saved to: {output_filename}")
print(f"✓ File contains {len(clean_df)} rows and {len(clean_df.columns)} columns")


print("\n" + "="*50)
print("CLEANING COMPLETE - SUMMARY")
print("="*50)
print(f"Original dataset rows: {len(df)}")
print(f"Cleaned dataset rows:  {len(clean_df)}")
print(f"Total rows removed:    {len(df) - len(clean_df)}")
print(f"Removal percentage:    {(len(df) - len(clean_df)) / len(df) * 100:.2f}%")
print("="*50)

# Optional: Show first few rows of cleaned data
if input("\nShow first 5 rows of cleaned data? (y/n): ").lower() == "y":
    print("\nFirst 5 rows of cleaned data:")
    print(clean_df.head())


OUTLIERS FOUND: Unrealistic Net_Metrekare (<10 or >2000)
Count: 47
      Net_Metrekare  Brüt_Metrekare  Oda_Sayısı Bulunduğu_Kat  Eşya_Durumu  \
187            3698          3700.0           4    Bilinmiyor            0   
1039              1             2.0           5  Yüksek Giriş            0   
1054              2             3.0           3        12.Kat            0   
1898           2661          2665.0           3    Bilinmiyor            0   
2003          12500         25000.0           6    Bilinmiyor            1   
2016          11308         11310.0           7    Bilinmiyor            0   
2089              5            40.0           2         1.Kat            0   
2353           4499          4500.0           8    Bilinmiyor            0   
3417          16500         17000.0           7    Bilinmiyor            1   
3577              2            45.0           2         2.Kat            0   

      Binanın_Yaşı       Isıtma_Tipi        Fiyat           Şehir  \
187 

In [9]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20131 entries, 0 to 20130
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Net_Metrekare       20131 non-null  int64  
 1   Brüt_Metrekare      20131 non-null  float64
 2   Oda_Sayısı          20131 non-null  int64  
 3   Bulunduğu_Kat       20131 non-null  object 
 4   Eşya_Durumu         20131 non-null  int64  
 5   Binanın_Yaşı        20131 non-null  float64
 6   Isıtma_Tipi         20131 non-null  object 
 7   Fiyat               20131 non-null  float64
 8   Şehir               20131 non-null  object 
 9   Binanın_Kat_Sayısı  20131 non-null  int64  
 10  Kullanım_Durumu     20131 non-null  int64  
 11  Yatırıma_Uygunluk   20131 non-null  int64  
 12  Takas               20131 non-null  int64  
 13  Tapu_Durumu         20131 non-null  object 
 14  Banyo_Sayısı        20131 non-null  float64
dtypes: float64(4), int64(7), object(4)
memory usage: 2.3+

Net_Metrekare         0
Brüt_Metrekare        0
Oda_Sayısı            0
Bulunduğu_Kat         0
Eşya_Durumu           0
Binanın_Yaşı          0
Isıtma_Tipi           0
Fiyat                 0
Şehir                 0
Binanın_Kat_Sayısı    0
Kullanım_Durumu       0
Yatırıma_Uygunluk     0
Takas                 0
Tapu_Durumu           0
Banyo_Sayısı          0
dtype: int64

In [ ]:
import re
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# ============================================================
# LOAD DATA
# ============================================================
df = pd.read_csv("home_price_cleaned.csv")

# ============================================================
# 1) ENCODE Binanın_Yaşı
# ============================================================
age_mapping = {
    "0 (Yeni)": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5-10": 7.5,
    "11-15": 13,
    "16-20": 18,
    "21 Ve Üzeri": 25
}

df["Binanın_Yaşı"] = df["Binanın_Yaşı"].map(age_mapping)
df["Binanın_Yaşı"] = df["Binanın_Yaşı"].fillna(df["Binanın_Yaşı"].median())
df["Binanın_Yaşı"] = df["Binanın_Yaşı"].astype(float)

# ============================================================
# 2) ENCODE Eşya_Durumu (binary)
# ============================================================
df["Eşya_Durumu"] = df["Eşya_Durumu"].map({
    "Boş": 0,
    "Eşyalı": 1
}).fillna(0).astype(int)

# ============================================================
# 3) ENCODE Takas (binary)
# ============================================================
df["Takas"] = df["Takas"].astype(str).str.strip().str.lower().map({
    "yok": 0,
    "var": 1
}).fillna(0).astype(int)

# ============================================================
# 4) ENCODE Kullanım_Durumu (binary)
# ============================================================
df["Kullanım_Durumu"] = df["Kullanım_Durumu"].map({
    "Boş": 0,
    "Mülk Sahibi Oturuyor": 0,
    "Kiracı Oturuyor": 1
}).fillna(0).astype(int)

# ============================================================
# 5) ENCODE Yatırıma Uygunluk (binary)
# ============================================================
df["Yatırıma_Uygunluk"] = df["Yatırıma_Uygunluk"].map({
    "Uygun": 1,
    "Bilinmiyor": 0,
    "bilinmiyor": 0
}).fillna(0).astype(int)

# ============================================================
# 6) LABEL ENCODE Şehir
# ============================================================
le_city = LabelEncoder()
df["Şehir"] = le_city.fit_transform(df["Şehir"].astype(str))

# ============================================================
# 7) LABEL ENCODE Isıtma Tipi
# ============================================================
le_heat = LabelEncoder()
df["Isıtma_Tipi"] = le_heat.fit_transform(df["Isıtma_Tipi"].astype(str))

# ============================================================
# 8) PARSE Bulunduğu Kat to numeric
# ============================================================

def extract_floor(x):
    if pd.isna(x):
        return None

    x = str(x).lower().strip()

    # Standard numeric floors: "4.kat"
    match = re.match(r"(\d+)\.kat", x)
    if match:
        return int(match.group(1))

    # 40+.Kat (treat as penthouse)
    if "40+.kat" in x:
        return 40

    # Kot floor: "Kot 1 (-2).Kat" → -2
    match_kot = re.match(r"kot\s*\d+\s*\(-(\d+)\)\.kat", x)
    if match_kot:
        return -int(match_kot.group(1))

    # Bodrum Kat → -1
    if "bodrum" in x:
        return -1

    # Bahçe Katı → 0
    if "bahçe katı" in x:
        return 0

    # Çatı Katı / Çatı Dubleks → top floor (approx)
    if "çatı" in x:
        return 30

    # Villa or Müstakil → no floor → assume 1
    if "villa" in x or "müstakil" in x:
        return 1

    # Yüksek Giriş → 0.5
    if "yüksek giriş" in x:
        return 0.5

    # Zemin/Giriş → 0
    if "zemin" in x or "giriş" in x:
        return 0

    return None

df["Bulunduğu_Kat"] = df["Bulunduğu_Kat"].apply(extract_floor)
df["Bulunduğu_Kat"] = df["Bulunduğu_Kat"].fillna(df["Bulunduğu_Kat"].median())
df["Bulunduğu_Kat"] = df["Bulunduğu_Kat"].astype(float)

# ============================================================
# SAVE CLEANED FILE
# ============================================================
df.to_csv("home_price_cleaned_first_encode.csv", index=False)
print("\nSaved as home_price_cleaned_first_encode.csv")

# Show summary
print("\nFinal Columns:")
print(df.dtypes)
print("\nNull Counts:")
print(df.isnull().sum())


In [ ]:
df = pd.read_csv("home_price_cleaned_first_encode.csv")

df["Tapu_Durumu"] = df["Tapu_Durumu"].map({
    "Bilinmiyor": 0,
    "Müstakil Tapulu": 1
})

df.to_csv("home_price_cleaned_first_encode.csv", index=False)
print("\nSaved as: home_price_cleaned_first_encode.csv")
